In [ ]:
# ================================
# Data Collection & Training (train.py)
# ================================
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, BatchNormalization, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from hmmlearn import hmm

# ----- CONFIG -----
SEQUENCE_LENGTH = 30
SIGN_CSV = 'sign_sequences.csv'
FACE_CSV = 'face_expressions.csv'

# ----- SIGN (Dynamic) DATA LOADING & PREP -----
df_sign = pd.read_csv(SIGN_CSV)
y_sign = df_sign['class'].values
X_sign = df_sign.drop('class', axis=1).values
# reshape to (num_samples, SEQUENCE_LENGTH, features_per_frame)
features_per_frame = X_sign.shape[1] // SEQUENCE_LENGTH
X_sign = X_sign.reshape(-1, SEQUENCE_LENGTH, features_per_frame)

# encode labels
le_sign = LabelEncoder()
y_sign_enc = le_sign.fit_transform(y_sign)

# split train/test
X_tr_s, X_te_s, y_tr_s, y_te_s = train_test_split(
    X_sign, y_sign_enc, test_size=0.3, random_state=1234, stratify=y_sign_enc
)

# scale per frame
scaler_sign = StandardScaler()
X_tr_flat = X_tr_s.reshape(-1, features_per_frame)
X_te_flat = X_te_s.reshape(-1, features_per_frame)
X_tr_scaled = scaler_sign.fit_transform(X_tr_flat).reshape(X_tr_s.shape)
X_te_scaled = scaler_sign.transform(X_te_flat).reshape(X_te_s.shape)

# ----- BUILD & TRAIN DEEPER LSTM -----
inputs = Input(shape=(SEQUENCE_LENGTH, features_per_frame), name='input')
# First LSTM stack (deeper network)
x = LSTM(256, return_sequences=True)(inputs)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
# Second LSTM layer
x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
# Third LSTM layer
x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
# Feature reducer
reduced = Dense(16, activation='linear', name='feature_reducer')(x)

# classification head
d1 = Dense(32, activation='relu')(reduced)
outputs = Dense(len(np.unique(y_sign_enc)), activation='softmax')(d1)

model_sign = Model(inputs, outputs)
model_sign.compile(
    optimizer=Adam(0.001), 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# callbacks for robust training
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3)

model_sign.fit(
    X_tr_scaled, y_tr_s,
    validation_data=(X_te_scaled, y_te_s),
    epochs=30, batch_size=8,
    callbacks=[early_stop, reduce_lr]
)

# ----- FEATURE EXTRACTION & GHMM -----
feat_extractor = Model(inputs, model_sign.get_layer('feature_reducer').output)
train_feat = feat_extractor.predict(X_tr_scaled)
test_feat = feat_extractor.predict(X_te_scaled)

# optional PCA
pca = PCA(n_components=16, random_state=42)
train_pca = pca.fit_transform(train_feat)
test_pca = pca.transform(test_feat)

# select best HMM model across dimensions
def select_best_n_components(X, n_range, criterion='bic'):
    best_score = np.inf
    best_model = None
    for n in n_range:
        try:
            m = hmm.GaussianHMM(
                n_components=n,
                covariance_type='diag',
                n_iter=100,
                tol=1e-3,
                init_params='stmc',
                verbose=False,
                random_state=42
            )
            m.fit(X)
            ll = m.score(X)
            k = (n * n) + 2 * n * X.shape[1] - 1
            bic = -2 * ll + k * np.log(len(X))
            if bic < best_score:
                best_score = bic
                best_model = m
        except Exception:
            continue
    return best_model

ghmm_sign = select_best_n_components(train_pca, range(1, 25))
if ghmm_sign is None:
    ghmm_sign = hmm.GaussianHMM(n_components=2,
        covariance_type='diag',
        n_iter=200,
        tol=1e-4,
        init_params='',
        random_state=42)
ghmm_sign.fit(train_pca)

# hidden states
hs_tr = ghmm_sign.predict(train_pca).reshape(-1, 1)
hs_te = ghmm_sign.predict(test_pca).reshape(-1, 1)
# combine
X_tr_rf = np.hstack((train_feat, hs_tr))
X_te_rf = np.hstack((test_feat, hs_te))

# random forest on combined features
rf_sign = RandomForestClassifier(n_estimators=150, max_depth=20, random_state=1234)
rf_sign.fit(X_tr_rf, y_tr_s)
y_pred_s = rf_sign.predict(X_te_rf)
print('Sign RF acc:', accuracy_score(y_te_s, y_pred_s))
print(classification_report(y_te_s, y_pred_s))

# ----- FACE (Static) DATA LOAD & TRAIN -----
df_face = pd.read_csv(FACE_CSV)
y_face = df_face['class'].values
X_face = df_face.drop('class', axis=1).values
le_face = LabelEncoder()
y_face_enc = le_face.fit_transform(y_face)
X_tr_f, X_te_f, y_tr_f, y_te_f = train_test_split(
    X_face, y_face_enc, test_size=0.3, random_state=1234, stratify=y_face_enc
)
scaler_face = StandardScaler()
X_tr_f_s = scaler_face.fit_transform(X_tr_f)
X_te_f_s = scaler_face.transform(X_te_f)

# train RF for face
rf_face = RandomForestClassifier(n_estimators=150, max_depth=15, random_state=1234)
rf_face.fit(X_tr_f_s, y_tr_f)
y_pred_f = rf_face.predict(X_te_f_s)
print('Face RF acc:', accuracy_score(y_te_f, y_pred_f))
print(classification_report(y_te_f, y_pred_f))

# ----- SAVE MODELS & ARTIFACTS -----
joblib.dump(le_sign, 'sign_label_encoder.pkl')
model_sign.save('sign_lstm_deeper.keras')
joblib.dump(ghmm_sign, 'sign_ghmm.pkl')
joblib.dump(rf_sign, 'sign_rf.pkl')
joblib.dump(scaler_sign, 'sign_scaler.pkl')
joblib.dump(pca, 'sign_pca.pkl')
joblib.dump(le_face, 'face_label_encoder.pkl')
joblib.dump(rf_face, 'face_rf.pkl')
joblib.dump(scaler_face, 'face_scaler.pkl')

